This is for a kaggle competition: https://www.kaggle.com/c/bosch-production-line-performance

In [ ]:
import xgboost
import pandas
import time
import io
import zipfile
import numpy
print('imports complete')

In [ ]:
# FYI: This takes about a minute and a half
def load_and_unzip(filename):
    print('loading and unzipping large training data file %s' % (filename))
    t0 = time.time()
    df = pandas.read_csv(filename, compression='zip')
    t1 = time.time()
    print('completed in %.3f seconds' % (t1-t0))
    return df

#train_numeric_filename = '../resources/train_numeric.csv.zip'
#df_explore = load_and_unzip(train_numeric_filename)
#test_numeric_filename = '../resources/test_numeric.csv.zip'
#df_test = load_and_unzip(test_numeric_filename)

Fill in the empty data with 0. get the mean and std of the entire training dataset, minus the id colum and the dependent variable column

In [ ]:
df_explore = pandas.read_csv('../resources/split_train_numeric_a.csv')
df_explore.fillna(0,inplace=True)
df_explore_meta = df_explore[df_explore.keys()[1:-1]].describe()
df_explore_meta

In [ ]:
def load_training_DMatrix(filename):
    arr = numpy.genfromtxt(filename, delimiter=',', skip_header=True, filling_values=0)
    return xgboost.DMatrix(arr[:,1:-1], label=arr[:,-1])
# load the testing file, without labels
def load_testing_DMatrix(filename):
    arr = numpy.genfromtxt(filename, delimiter=',', skip_header=True, filling_values=0)
    return arr[:,0], xgboost.DMatrix(arr[:,1:])

In [ ]:
dtrain = load_training_DMatrix('../resources/train_numeric.csv')
param = {'max_depth':2, 'eta':1, 'slient':1, 'objective':'binary:logistic'}
num_round = 10
bst = xgboost.train(param, dtrain, num_round)

In [ ]:
ids, dtest = load_testing_DMatrix('../resources/test_numeric.csv')

In [ ]:
pred = bst.predict(dtest)

In [ ]:
with open('default_xgboost_results.txt', 'w') as f:
    f.write('Id,Response\n')
    for i, p in enumerate(pred):
        f.write('%d,%s\n'%(ids[i],'1' if p > 0.5 else '0'))

use the xgboost classifier
cross validation with k fold
grid search over a few parameters
data stored in pandas DataFrame
still only using just the numeric data

In [1]:
import sklearn.metrics
import sklearn.grid_search
import sklearn.cross_validation
import xgboost.sklearn
import pandas

In [2]:
data = pandas.read_csv('../resources/train_numeric.csv')
data.fillna(0,inplace=True)
target_name = 'Response'
target = data[target_name]
features = data.drop(target_name, axis=1)

In [8]:
def perf_metric(y_true, y_predict):
    return sklearn.metrics.matthews_corrcoef(y_true, y_predict)

def fit_model(X,y):
    cv_sets = sklearn.cross_validation.ShuffleSplit(X.shape[0], n_iter=10, test_size=0.1, random_state=0)
    clf = xgboost.sklearn.XGBClassifier(nthread=14)
    params = {
                'max_depth' : [6],
                'reg_lambda' : [1]
             }
    scoring_fnc = sklearn.metrics.make_scorer(perf_metric)
    grid = sklearn.grid_search.GridSearchCV(clf, params, scoring=scoring_fnc, cv=cv_sets, verbose=2)
    grid = grid.fit(X,y)
    return grid.best_estimator_

clf = fit_model(features, target)

Fitting 10 folds for each of 1 candidates, totalling 10 fits
[CV] max_depth=6, reg_lambda=1 .......................................
[CV] .............................. max_depth=6, reg_lambda=1 - 4.5min
[CV] max_depth=6, reg_lambda=1 .......................................
[CV] .............................. max_depth=6, reg_lambda=1 - 4.4min
[CV] max_depth=6, reg_lambda=1 .......................................
[CV] .............................. max_depth=6, reg_lambda=1 - 4.4min
[CV] max_depth=6, reg_lambda=1 .......................................
[CV] .............................. max_depth=6, reg_lambda=1 - 4.4min
[CV] max_depth=6, reg_lambda=1 .......................................
[CV] .............................. max_depth=6, reg_lambda=1 - 4.4min
[CV] max_depth=6, reg_lambda=1 .......................................
[CV] .............................. max_depth=6, reg_lambda=1 - 4.4min
[CV] max_depth=6, reg_lambda=1 .......................................
[CV] ...........

[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed: 44.4min finished


In [9]:
clf.get_params()

{'base_score': 0.5,
 'colsample_bylevel': 1,
 'colsample_bytree': 1,
 'gamma': 0,
 'learning_rate': 0.1,
 'max_delta_step': 0,
 'max_depth': 6,
 'min_child_weight': 1,
 'missing': None,
 'n_estimators': 100,
 'nthread': 14,
 'objective': 'binary:logistic',
 'reg_alpha': 0,
 'reg_lambda': 1,
 'scale_pos_weight': 1,
 'seed': 0,
 'silent': True,
 'subsample': 1}

In [10]:
test_data = pandas.read_csv('../resources/test_numeric.csv')
pred = clf.predict(test_data)
with open('xgboost_all_numeric_results.txt', 'w') as f:
    f.write('Id,Response\n')
    for i, p in enumerate(pred):
        f.write('%d,%d\n'%(test_data['Id'][i],p))